In [1]:
import cv2

from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

model = YOLO("./weights/yolo11x.pt")
names = model.names

cap = cv2.VideoCapture("./data/video3.mp4")

In [4]:
from PIL import Image

In [2]:
cap.set(1, 2)
res, frame = cap.read() #frame has your pixel values

#Get frame height and width to access pixels
height, width, channels = frame.shape

In [3]:
frame

array([[[ 55,  57,  59],
        [ 58,  60,  62],
        [ 62,  64,  66],
        ...,
        [153, 159, 163],
        [153, 159, 163],
        [153, 159, 163]],

       [[ 54,  56,  58],
        [ 57,  59,  61],
        [ 62,  64,  66],
        ...,
        [153, 159, 163],
        [153, 159, 163],
        [153, 159, 163]],

       [[ 49,  51,  53],
        [ 54,  56,  58],
        [ 58,  60,  62],
        ...,
        [150, 156, 160],
        [150, 156, 160],
        [150, 156, 160]],

       ...,

       [[192, 199, 200],
        [193, 200, 201],
        [195, 202, 203],
        ...,
        [ 34,  34, 181],
        [ 33,  33, 180],
        [ 33,  33, 180]],

       [[193, 200, 201],
        [194, 201, 202],
        [196, 203, 204],
        ...,
        [ 34,  34, 181],
        [ 34,  34, 181],
        [ 33,  33, 180]],

       [[193, 200, 201],
        [195, 202, 203],
        [196, 203, 204],
        ...,
        [ 34,  34, 181],
        [ 34,  34, 181],
        [ 33,  33, 180]]

In [ ]:
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

In [3]:
w, h, fps

(1080, 1920, 29)

In [ ]:
# Blur ratio
blur_ratio = 50

# Video writer
video_writer = cv2.VideoWriter("object_blurring_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    results = model.predict(im0, show=False)
    boxes = results[0].boxes.xyxy.cpu().tolist()
    clss = results[0].boxes.cls.cpu().tolist()
    annotator = Annotator(im0, line_width=2, example=names)

    if boxes is not None:
        for box, cls in zip(boxes, clss):
            annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

            obj = im0[int(box[1]) : int(box[3]), int(box[0]) : int(box[2])]
            blur_obj = cv2.blur(obj, (blur_ratio, blur_ratio))

            im0[int(box[1]) : int(box[3]), int(box[0]) : int(box[2])] = blur_obj

    cv2.imshow("ultralytics", im0)
    video_writer.write(im0)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
video_writer.release()
cv2.destroyAllWindows()